# Travel Itinerary Planner - Trip Designer


### Workflow overview :

1. Install all the prerequisites
2. Defining the search tool to retrive data related to the destination
3. Create agents for data gathering, data analysis, and recommendation
4. Configure tasks aligning with agents roles and objectives
5. Set up a Crew to perform these tasks in a coordinated sequence, enabling effective collaboration between the agents
6. Kickoff the crew and observe the results



### Step 1 : Install all the prerequisities

In [15]:
# Prerequisities - Installations

!pip install crewai
!pip install crewai-tools
!pip install duckduckgo-search

### Step 2 : Defining the search tool to retrive data related to the destination

In [16]:
# Importing necessary libraries
from crewai import Agent, Task
from crewai.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
from datetime import datetime

# Setting the date and time for inputs
Now = datetime.now()
Current_day = Now.strftime("%d-%b-%Y")

# Defining the search tool for data gathering agent
@tool("DuckDuckGo Search")
def websearch(search_query: str):
    """ Performing a web-based query to gather topic-specific information """
    return DuckDuckGoSearchRun().run(search_query)

### Step 3 : Create agents for data gathering, data analysis, and recommendation

In [17]:
# Defining the Agents

data_gatherer = Agent(
    role = 'Gathers the data',
    goal = 'Gathers latest data with respect to hotels , restaurants , tourist spots and all other relevant data required for a traveller',
    llm='gpt-4o-mini',
    verbose=True,
    backstory = (
        'You are an expert data explorer and gatherer, who can gather all travel and tourist spot related latest information. Considering the current date : ' + Current_day
    ),
    tools=[websearch],
    cache=True,
    max_iter=4
)


data_analyser = Agent(
    role = 'Precisely analyses the data',
    goal = 'Consolidate and analyse the provided data',
    llm='gpt-4o-mini',
    verbose=True,
    backstory = (
        'You are an expert in analysing the data provided.Considering the current date : ' + Current_day
    ),

)

recommender = Agent(
    role = 'Recommends and suggests the travel plan',
    goal = 'Considering the inputs and analyzed data provided , you make travel itinerary . Suggest food spots , must-visits and travel tips ',
    llm='gpt-4o-mini',
    verbose=True,
    backstory = (
        'You are an travel agent expert in recommending the itinerary considering the detination , budget, days available and interests . Considering the current date : ' + Current_day
    ),
)

### Step 4 : Configure tasks aligning with agents roles and objectives

In [18]:
# Configuring the tasks

gather_the_data = Task(
    description = "Collect end to end travel related data for the place: {destination} required for a travel guide",
    expected_output = "Latest end to end travel data including all hotels , restaurants , mustvisits , mustdos and all other things related to place",
    agent = data_gatherer,
)

analyse_the_data = Task(
    description = "Make extensive analysis based on provided travel information of the place : { destination }",
    expected_output = "Comprehensive and thorough analysis of the provided travel information related to the place : { destination } ",
    agent = data_analyser,
    context = [gather_the_data],
    output_file = "Analysis_travel.txt"
)

recommend_the_plan = Task(
    description = "Recommends best suitable itinerary plans for morning , afternoon , evening and night , food , restaurant and tourist spots , must visits , mustdo's and other travel tips"
                        "based on place : {destination} , number of available days : {days} ,traveller interests : {interests} and budget : {budget}",
    expected_output = "Make a recommendation about best itinerary , travel plans and other required suggestions according to provided inputs",
    agent = recommender,
    context = [analyse_the_data],
    output_file = "Recommendation.txt"
)

### Step 5 : Set up a Crew to perform these tasks in a coordinated sequence

In [19]:
# Initializing the Crew

from crewai import Crew, Process

# Callback function to print a timestamp
def timestamp(Input):
    print(datetime.now())

# Define the crew with agents and tasks in sequential process
crew = Crew(
    agents=[data_gatherer, data_analyser, recommender],
    tasks=[gather_the_data, analyse_the_data, recommend_the_plan],
    verbose=False,
    Process=Process.sequential,
    step_callback=timestamp,
)

### Step 6 : Kickoff the crew and observe the results

In [20]:
# Running the Crew

from google.colab import userdata
import os

# Set your Groq API key or openAI API key
os.environ['OPENAI_API_KEY'] = userdata.get('PENAI_1')

# Run the crew with the specific inputs
results = crew.kickoff(inputs={'destination': 'Thailand', 'days': 6 , 'budget': 90000 , 'interests' : 'nature'})

print("----------------------------------------------------------")
print("Final Result:", results)

# Agent: Gathers the data
## Task: Collect end to end travel related data for the place: Thailand required for a travel guide
2025-05-23 17:16:48.269628


# Agent: Gathers the data
## Thought: I need to gather comprehensive travel-related information about Thailand, including details on hotels, restaurants, must-visit places, and activities to create an informative travel guide.
## Using tool: DuckDuckGo Search
## Tool Input: 
"{\"search_query\": \"latest travel guide Thailand hotels restaurants tourist attractions must visit May 2025\"}"
## Tool Output: 
Check out this Thailand Travel Guide 2025 before you book your flight! Get the best tips on places to visit, budgets, safety, and travel hacks for an epic trip. ... A must-see is the Grand Palace, the former home of the Thai kings. ... and All the Latest Details Revealed. May 7, 2025; Travel Top 10 Most Beautiful Lakes on Earth That Are Worth ... Koh Samui - Island life. As one of the most popular tourist destinations in Thailand, Koh